In [24]:
!pip install beautifulsoup4




In [84]:
import requests
from bs4 import BeautifulSoup
import re
import json

from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_mail(sender_email,receiver_email, subject, content):
    import smtplib, ssl


    message = MIMEMultipart("alternative")
    message["Subject"] = subject
    message["From"] = sender_email
    message["To"] = receiver_email
    
#     password = input("Type your password and press enter:")

    # Create the plain-text and HTML version of your message
    text = content
    html = """\
    <html>
      <body>
        <p>Hi,<br>
           How are you?<br>
           <a href="http://www.realpython.com">Real Python</a> 
           has many great tutorials.
        </p>
      </body>
    </html>
    """

    # Turn these into plain/html MIMEText objects
    part1 = MIMEText(text, "plain")
#     part2 = MIMEText(html, "html")

    # Add HTML/plain-text parts to MIMEMultipart message
    # The email client will try to render the last part first
    message.attach(part1)
#     message.attach(part2)

    # Create secure connection with server and send email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        password = "dbckremrlmngbbhg"
        server.login(sender_email, password)
        server.sendmail(
            sender_email, receiver_email, message.as_string()
        )

def fetch_the_script_content(url, info):
    try:
        page = requests.post(url, data=info)
    except:
        sender_email = "shilpaar90@gmail.com"
        receiver_email = "shilpaar90@gmail.com"
        password = "dbckremrlmngbbhg"
        subject = "Slot finder update - Not reachable" 
        send_mail(sender_email, receiver_email, subject,'' )
    soup = BeautifulSoup(page.content, 'html.parser')
    table_contents = soup.find_all('script',string='')
    return table_contents[-1]

def find_pattern_in_string(pattern, string):
    string_results = re.findall(pattern, string)
#     print("String results", string_results)
    details = []
    for string_result in string_results:
        center_info_json = json.loads(string_result)
        for center_detail in center_info_json:
            details.append(dict(zip(["Center Name","Area","Pincode","Earliest Date","Slots"],center_detail)))
        return details

def get_vaccine_center_info(url,info):
    test_data = fetch_the_script_content(url,info )
    pattern = re.compile('\[\[".*",".*",.*,".*",[0-9]*\]\]')
    center_details = find_pattern_in_string(pattern, test_data.string)
    return (center_details)


In [85]:
url = "https://under45.in/result.php"
chennai_data = {"state":"31","district":"571","stateName":"Tamil+Nadu","districtName":"Chennai"}
coimbatore_data = {"state":"31","district":"539","stateName":"Tamil+Nadu","districtName":"Coimbatore"}

odisha_khurda_data = {"state":"26","district":"446","stateName":"Odisha","districtName":"Khurda"}
maha_nanded = {"state":"21","district":"382","stateName":"Maharashtra","districtName":"Nanded"}

bangalore_urban_data = {"state":"16","district":"265","stateName":"Karnataka","districtName":"Bangalore Urban"}
bangalore_rural_data = {"state":"16","district":"276","stateName":"Karnataka","districtName":"Bangalore Rural"}
karnataka_bbmp_data = {"state":"16","district":"294","stateName":"Karnataka","districtName":"BBMP"}
gujarat_ahmedabad_corp_data = {"state":"11","district":"770","stateName":"Gujarat","districtName":"Ahmedabad Corporation"}


In [86]:

sender_email = "shilpaar90@gmail.com"
receiver_email = "shilpaar90@gmail.com"
password = "dbckremrlmngbbhg"
subject = "Slot finder update -" 

In [87]:
import time
import pandas as pd

def monitor_slots(region_data):
    prev_content = 'None'
    while True:
        content = get_vaccine_center_info(url, region_data)
#         print(content)
        if content:
            if prev_content and content != prev_content:
                send_mail(sender_email, receiver_email, subject + region_data['districtName'] + ' - ' + str(pd.DataFrame(content)['Center Name'].count()) + ' centers, ' + str(pd.DataFrame(content)['Slots'].sum()) + ' slots', json.dumps(content, indent=2))
                prev_content = content
        time.sleep(2*10)

In [ ]:
monitor_slots(bangalore_urban_data)